Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
import matplotlib.pyplot as plt
import pprint
import numpy as np
import seaborn as sns
import csv
import random
from selenium.webdriver.chrome.options import Options
import re
import os

Getting the list of course IDs 

In [ ]:

# Define the range of years
start_year = 2017
end_year = 2024

# Example list of department codes (replace these with actual department codes)
departments = ['1', '10', '11', '12', '13', '22', '23', '24', '25', '26', '27', '28', '29', '30', '33', '34', '36', '38', '41', '42', '46', '47', '56', '62', '63', '83', '88', 'CB', 'KU']
course_codes = set()  # Using a set to automatically handle duplicates

# Loop over each academic year
for year in range(start_year, end_year + 1):
    academic_year = f"{year}-{year+1}"
    
    # Loop over each department code
    for dept in departments:
        url = f"https://kurser.dtu.dk/archive/{academic_year}/department/{dept}"
        
        # Fetch the page content
        response = requests.get(url, headers={'name':'Students from UCPH','email':'lqz683@alumni.ku.dk'})
        if response.status_code == 200:  # Proceed only if the page exists
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all course codes (assuming they are in <td> tags inside a table row <tr>)
            rows = soup.find_all('tr')

            for row in rows:
                cells = row.find_all('td')
                if cells:
                    # Assuming the course code is in the first cell (td)
                    course_code = cells[0].get_text(strip=True)
                    course_codes.add(course_code)  # Add to the set to avoid duplicates
                     
   # Pause before the next request
        time.sleep(0.5)  # Sleep for 0.5 seconds 

# Convert the set to a sorted list
course_codes_list = sorted(course_codes)

# Create a DataFrame for better visualization and export
df = pd.DataFrame(course_codes_list, columns=["Course Code"])

# Display the first few rows to check
print(df.head())

# Save to Excel for further analysis
#df.to_excel('course_codes.xlsx', index=False)

Getting the course information from the descriptions on kurser.dtu.dk by scraping with Selenium

In [ ]:
def fetch_kursusinformation_with_selenium(driver, semester, course_number):
    url = f"https://kurser.dtu.dk/course/{semester}/{course_number}"
    driver.get(url)
    
    time.sleep(random.uniform(1.08, 1.44))  # Give it a moment to load fully if necessary
    
    # Get the page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Find the specific "Kursusinformation" table or section within the div
    kursusinformation_section = soup.select_one("div.box.information")
    
    if kursusinformation_section:
        # Extract all the text within the Kursusinformation section
        kursusinformation_text = kursusinformation_section.get_text(separator="\n", strip=True)
        return kursusinformation_text
    else:
        return None  # Return None if no information is found

def save_kursusinformation_to_csv(semester, course_number, kursusinformation_text, filename="kursusinformation.csv"):
    # Write Kursusinformation data to a CSV file
    with open(filename, mode='a', newline='', encoding='utf-8') as file:  # Use 'a' mode to append to the file
        writer = csv.writer(file)
        
        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(["Semester", "Course Number", "Kursusinformation"])
        
        # Write the Kursusinformation with the semester and course number
        writer.writerow([semester, course_number, kursusinformation_text])

# List of semesters from 2017-2018 to 2023-2024 (we split the range for more time flexability, but the end result is the same)
semesters = [f"{year}-{year+1}" for year in range(2023, 2024)]

# Generate the first 20 course numbers between 01000 to 88718
course_numbers = df['Course Code'].tolist()

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Optional: Start maximized
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model

# Start the browser session once
driver = webdriver.Chrome(options=chrome_options)

try:
    # Loop through each semester and the first 20 course numbers and scrape data
    for semester in semesters:
        for course_number in course_numbers:
            kursusinformation_text = fetch_kursusinformation_with_selenium(driver, semester, course_number)
            if kursusinformation_text:
                print(f"Fetched Kursusinformation for {semester} - {course_number}")
                # Save the extracted Kursusinformation to a CSV file
                save_kursusinformation_to_csv(semester, course_number, kursusinformation_text, filename="kursusinformation.csv")
finally:
    # Close the browser session when done
    driver.quit()

print("Data fetching complete. Results saved to kursusinformation.csv")

Getting the data for GPA statistics from karakterer.dtu.dk. 

In [ ]:
# Define the years and periods
years = range(2017, 2024+1)  # Adjust as needed
seasons = ['Summer', 'Winter']

# Prepare the CSV file path
output_file_path = './combined_course_data2.csv'

# Check if the file already exists; if not, create it with headers
if not os.path.exists(output_file_path):
    with open(output_file_path, 'w') as f:
        # Writing header for the first time
        f.write('Course,Year,Season,Column1,Column2,...\n')  # Adjust columns as needed

# Loop through each course number, year, and season
for course in course_codes:  # Assuming you're using the list course_codes from the Excel file
    for year in years:
        for season in seasons:
            # Construct the URL dynamically
            period = f'{season}-{year}'
            url = f'https://karakterer.dtu.dk/Histogram/1/{course}/{period}'
            
            try:
                # Parse all tables found on the page
                dfs = pd.read_html(url)
                
                # Check if any tables were found
                if dfs:
                    # Assuming the first table is the one you want
                    df = dfs[0]
                    
                    # Add additional columns to the DataFrame for course, year, and season
                    df.insert(0, 'Course', course)
                    df.insert(1, 'Year', year)
                    df.insert(2, 'Season', season)
            
                    
                    # Append the DataFrame to the CSV file
                    df.to_csv(output_file_path, mode='a', header=False, index=False)
                    
                    print(f"Data for {course} during {period} added.")
                else:
                    print(f"No data available for {course} during {period}. Skipping...")
            
            except ValueError:
                # Handle the case where no tables are found on the page
                print(f"No tables found for {course} during {period}. Skipping...")
            except Exception as e:
                print(f"An error occurred while processing {course} during {period}: {e}")

            # Add a delay between each request for different year/season combinations
            time.sleep(0.5)  # Pause for 0.5 second 

print(f"Data collection completed. Data saved to '{output_file_path}'.")

Changing the GPA data into a dataframe to inspect it

In [ ]:
# Path to the CSV file
csv_file = './combined_course_data2.csv'

# Load the CSV file into a DataFrame
df_gpa = pd.read_csv(csv_file)

# Assuming df is the DataFrame you're working with
# Step 1: Pivot the DataFrame
df_pivot = df_gpa.pivot_table(index=['Course', 'Year', 'Season'], columns='Column1', values='Column2', aggfunc='first').reset_index()

# Step 2: Display the transformed DataFrame
print(df_pivot.head)

Drop unnecesary data

In [ ]:
# Drop the "Andre versioner" column
df_pivot = df_pivot.drop(columns=['Andre versioner'])

Change GPA data back into CSV file

In [ ]:
# Save the pivoted DataFrame to a new CSV file
output_csv_path = './pivoted_course_data.csv'
df_pivot.to_csv(output_csv_path, index=False)

# Step 2: Save the pivoted DataFrame to an Excel file
output_excel_path = './pivoted_course_data.xlsx'
df_pivot.to_excel(output_excel_path, index=False)

Change course information data into a data frame to inspect and edit and change it back again

In [ ]:

# Load the original CSV file into a DataFrame with specified headers
csv_file_path = './kursusinformation.csv'  # Update the path as necessary
df = pd.read_csv(csv_file_path, header=None, names=['Year', 'CourseCode', 'Kursusinformation'])

# Define a function to add space between line shifts
def add_space_to_line_shifts(text):
    # Replace line shifts (newline characters) with a space followed by the newline character
    return text.replace('\n', ' ')

# Apply the function to the "Kursusinformation" column
df['Kursusinformation'] = df['Kursusinformation'].apply(add_space_to_line_shifts)

# Display the first few rows of the modified DataFrame to verify the changes
print(df.head())

# Save the modified DataFrame back to a CSV file or proceed with further processing
output_csv_path = './kursusinformation.csv'
df.to_csv(output_csv_path, index=False)


Edit observations

In [ ]:
def split_kursusinformation(info):
    # Define the keywords for splitting, including "Institut"
    keywords = [
        "Engelsk titel", "Undervisningssprog", "Point( ECTS )", "Kursustype",
        "Undervisningens placering", "Undervisningsform", "Kursets varighed", 
        "Eksamensplacering", "Evalueringsform", "Eksamens varighed",  
        "Hjælpemidler", "Bedømmelsesform", "Pointspærring", 
        "Anbefale faglige forudsætninger", "Anbefalede forudsætninger", 
        "Kursusansvarlig", "Medansvarlige", "Institut",
        "Ekstern samarbejdsinstitution", 
        "Tilmelding", "Mulighed for GRØN DYST deltagelse", 
        "Deltagende institut", "Skemaplacering", "Deltagerbegrænsning", 
        "Tidligere kursus", "Obligatoriske forudsætninger"
    ]

    result = {}
    
    pattern = '|'.join([re.escape(kw) for kw in keywords])
    matches = list(re.finditer(pattern, info))
    
    # Collect non-Institut data
    for i, match in enumerate(matches):
        start_keyword = match.group().strip()
        start_pos = match.end()

        if i + 1 < len(matches):
            end_pos = matches[i + 1].start()
        else:
            end_pos = len(info)

        value = info[start_pos:end_pos].strip()
        result[start_keyword] = value

    return result

def extract_institut(info):
    # Extract every "Institut" occurrence and the text that follows it
    segments = re.split(r'(Institut)', info)
    
    institut_data = []
    for i in range(len(segments) - 1):
        if segments[i] == 'Institut':
            institut_data.append('Institut' + segments[i + 1].strip())

    return ' '.join(institut_data).strip()

# Load the DataFrame
csv_file_path = './kursusinformation.csv'
df = pd.read_csv(csv_file_path, header=None, names=['Year', 'CourseCode', 'Kursusinformation'])

# Split data without handling "Institut"
split_data = df['Kursusinformation'].apply(split_kursusinformation)
split_df = pd.json_normalize(split_data)

# Handle "Institut" separately
split_df['Institut'] = df['Kursusinformation'].apply(extract_institut)

# Combine the original columns with the new split columns
final_df = pd.concat([df[['Year', 'CourseCode']], split_df], axis=1)

# Save the final DataFrame to a CSV file
output_csv_path = './final_split_kursusinformation.csv'
final_df.to_csv(output_csv_path, index=False)

# Save the final DataFrame to an Excel file
output_excel_path = './final_split_kursusinformation.xlsx'
final_df.to_excel(output_excel_path, index=False)

**MERGE**

In [ ]:
grades = pd.read_csv('./pivoted_course_data.csv')  
info = pd.read_csv('./final_split_kursusinformation.csv')


print(grades.head())
print(info.head())

Edit dataframe

In [ ]:
# Rename 'Course' to 'CourseCode' in grades
grades.rename(columns={'Course': 'Course code'}, inplace=True)

# Rename 'Year' to 'Academic Year' in info
info.rename(columns={'CourseCode': 'Course code', 'Year': 'Academic Year'}, inplace=True)

# Display the datasets after renaming
print(grades.head())
print(info.head())

Create new column

In [ ]:
# Create an 'Academic Year' column in grades based on 'Year' and 'Season'
def create_academic_year(row):
    if row['Season'].lower() == 'winter':
        return f"{row['Year']}-{row['Year'] + 1}"
    elif row['Season'].lower() == 'summer':
        return f"{row['Year'] - 1}-{row['Year']}"
    else:
        return None

grades['Academic Year'] = grades.apply(create_academic_year, axis=1)

# Display the modified grades dataset to verify the new 'Academic Year' column
print(grades[['Year', 'Season', 'Academic Year']].head())

In [ ]:
# Merge the datasets on 'Course code' and 'Academic Year'
merged_df = pd.merge(grades, info, on=['Course code', 'Academic Year'], how='inner')

# Reorder columns so that 'Course code' and 'Academic Year' are the first two
desired_order = ['Course code', 'Academic Year'] + [col for col in merged_df.columns if col not in ['Course code', 'Academic Year']]
merged_df = merged_df[desired_order]

# Save the merged dataset to a new CSV file
output_csv_path = './merged_course_data.csv'
merged_df.to_csv(output_csv_path, index=False)

# Save the merged dataset to an Excel file
output_excel_path = './merged_course_data.xlsx'
merged_df.to_excel(output_excel_path, index=False)

# Display a message indicating where the files are saved
print(f"Merged data saved to '{output_csv_path}' and '{output_excel_path}'.")

***Clean data***

In [ ]:
# Get the number of rows before removing duplicates
rows_before = merged_df.shape[0]

# Remove duplicate rows from the entire DataFrame
merged_df.drop_duplicates(inplace=True)

# Get the number of rows after removing duplicates
rows_after = merged_df.shape[0]

# Print the number of rows before and after
print(f"Number of rows before removing duplicates: {rows_before}")
print(f"Number of rows after removing duplicates: {rows_after}")

In [ ]:
# Drop the specified columns
columns_to_drop = ["Kursusansvarlig", "Medansvarlige", "Tilmelding", "Pointspærring", "Skemaplacering", 
"Eksamensplacering", "Deltagerbegrænsning", "Tidligere kursus", "Deltagende institut",
"Undervisningens placering", "Undervisningsform", "Kursets varighed", "Eksamens varighed"]
clean_merged_df = merged_df.drop(columns=columns_to_drop)

# Save the final DataFrame to a CSV file
cleaned_csv_path = './cleaned_merged_course_data.csv'
clean_merged_df.to_csv(cleaned_csv_path, index=False)

# Save the final DataFrame to an Excel file
cleaned_excel_path = './cleaned_merged_course_data.xlsx'
clean_merged_df.to_excel(cleaned_excel_path, index=False)

In [ ]:

# Ensure 'Ekstern samarbejdsinstitution' is treated as a string for processing
clean_merged_df['Ekstern samarbejdsinstitution'] = clean_merged_df['Ekstern samarbejdsinstitution'].astype(str)

# Set 'Ekstern samarbejdsinstitution' to "Nej" if blank, NaN, or contains only whitespace, otherwise "Ja"
clean_merged_df['Ekstern samarbejdsinstitution'] = np.where(
    (clean_merged_df['Ekstern samarbejdsinstitution'].str.strip().isin(['', 'nan'])), 
    'Nej', 
    'Ja'
)

In [ ]:
# Combine 'Anbefale faglige forudsætninger', 'Anbefalede forudsætninger', and 'Obligatoriske forudsætninger' into 'forudsætninger'
clean_merged_df['forudsætninger'] = (
    clean_merged_df['Anbefale faglige forudsætninger'].fillna('') + ' ' +
    clean_merged_df['Anbefalede forudsætninger'].fillna('') + ' ' +
    clean_merged_df['Obligatoriske forudsætninger'].fillna('')
).str.strip()  # .str.strip() removes any leading or trailing whitespace

# Set 'forudsætninger' to "Nej" if blank, NaN, or contains "Ingen"; otherwise "Ja"
clean_merged_df['forudsætninger'] = np.where(
    (clean_merged_df['forudsætninger'].str.strip() == '') |
    (clean_merged_df['forudsætninger'].str.contains("Ingen", case=False, na=False)),
    'Nej', 
    'Ja'
)

# Drop the old columns 'Anbefale faglige forudsætninger', 'Anbefalede forudsætninger', 'Obligatoriske forudsætninger'
clean_merged_df.drop(columns=['Anbefale faglige forudsætninger', 'Anbefalede forudsætninger', 'Obligatoriske forudsætninger'], inplace=True)

In [ ]:
# Step 1: Remove rows where "Eksamensgennemsnit" is NaN
clean_merged_df = clean_merged_df.dropna(subset=['Eksamensgennemsnit'])

# Step 2: Remove rows where "Eksamensgennemsnit" is "Intet eksamensgennemsnit"
clean_merged_df = clean_merged_df[clean_merged_df['Eksamensgennemsnit'] != 'Intet eksamensgennemsnit']

# Remove the text "(Efter 7-trinsskalaen)" from the "Eksamensgennemsnit" column
clean_merged_df['Eksamensgennemsnit'] = clean_merged_df['Eksamensgennemsnit'].str.replace(' \(Efter 7-trinsskalaen\)', '', regex=True)

print(f"Number of rows after cleaning for Eksamensgennemsnit: {len(clean_merged_df)}")

In [ ]:
# Define the allowed values
allowed_values = ["Kandidat", "Bachelor", "Ph.d.", "Diplomingeniør", "Deltidsmaster"]

# Function to filter and retain only allowed values in Kursustype
def clean_kursustype(text):
    # Split the text into words and filter only allowed values
    cleaned_words = [word for word in allowed_values if word in text]
    
    # Join the matched words with a comma and space, or return NaN if no matches
    return ', '.join(cleaned_words) if cleaned_words else pd.NA

# Apply the function to the Kursustype column
clean_merged_df['Kursustype'] = clean_merged_df['Kursustype'].apply(clean_kursustype)

# Drop rows where Kursustype is NaN (i.e., no allowed values were found)
clean_merged_df = clean_merged_df.dropna(subset=['Kursustype'])

print(f"Number of rows after cleaning for Kursustype: {len(clean_merged_df)}")

In [ ]:
# Function to clean Antal bestået by removing text in parentheses
def clean_antal_bestaet(text):
    # Use regular expressions to keep only the number before any parentheses
    cleaned_text = re.sub(r'\s*\(.*?\)', '', text).strip()
    return cleaned_text

# Apply the function to the Antal bestået column
clean_merged_df['Antal bestået'] = clean_merged_df['Antal bestået'].apply(clean_antal_bestaet)

# Display the first few rows to verify the changes
print(f"Number of rows after cleaning for Antal bestået: {len(clean_merged_df)}")

In [ ]:
# Function to clean the 'Bedømmelsesform' column by replacing entire lines with "Intern" or "Ekstern"
def clean_bedommelsesform(text):
    # Simplify text to "Intern" if it contains "intern"
    if 'intern' in text.lower():
        return 'Intern'
    
    # Simplify text to "Ekstern" if it contains "ekstern"
    if 'ekstern' in text.lower():
        return 'Ekstern'
    
    # If neither "intern" nor "ekstern" is found, return the text unchanged
    return text.strip()

# Apply the function to the 'Bedømmelsesform' column
clean_merged_df['Bedømmelsesform'] = clean_merged_df['Bedømmelsesform'].apply(clean_bedommelsesform)

# Display the first few rows to verify the changes
print(f"Number of rows after cleaning for Bedømmelsesform: {len(clean_merged_df)}")

In [ ]:
# Function to map the existing values to the desired new values
def map_gron_dyst(value):
    if pd.isna(value):  # Check if the value is missing (NaN)
        return "Ikke oplyst"
    elif "Dette kursus giver den studerende en mulighed" in value:
        return "Ja"
    elif "Kontakt underviseren for information" in value:
        return "Måske"
    else:
        return value  # Return the original value if no match is found

# Apply the function to the Mulighed for GRØN DYST deltagelse column
clean_merged_df['Mulighed for GRØN DYST deltagelse'] = clean_merged_df['Mulighed for GRØN DYST deltagelse'].apply(map_gron_dyst)

# Display the first few rows to verify the changes
print(f"Number of rows after cleaning for Mulighed for GRØN DYST deltagelse: {len(clean_merged_df)}")

In [ ]:
# Define a function to process the text
def process_institut_text(text):
    # Check if the line contains 'Københavns Universitet' and replace it entirely
    if "Københavns Universitet" in text:
        return "88 Andre kurser"
    
    # Remove everything before and including an email address directly in front of 'Institut' if it exists
    email_patterns = [r'\S+@[\w.-]+\.dk', r'\S+@[\w.-]+\.com', r'\S+@[\w.-]+\.no']
    for pattern in email_patterns:
        # Use regex to find an email directly in front of the text 'Institut'
        match = re.search(rf'{pattern}\s+Institut', text)
        if match:
            email_start = match.start()
            text = text[email_start + len(match.group()):].strip()  # Keep everything after the email address and 'Institut'
            break  # Only process the first email found

    # Remove everything before and including 'Simonsen' if it exists
    if "Simonsen" in text:
        text = text.split("Simonsen", 1)[-1].strip()  # Keep everything after 'Simonsen' and strip extra spaces
    
    # Define the keywords for truncation
    keywords = ['Kursushjemmeside', 'Tilmelding', 'Deltagende', 'Ekstern']
    
    # Find the position of the first occurrence of any keyword
    positions = [text.find(keyword) for keyword in keywords if keyword in text]
    
    # If any keyword is found, truncate the text from the first occurrence of the keyword
    if positions:
        min_pos = min(positions)
        text = text[:min_pos].strip()  # Return the text up to but not including the keyword and strip extra spaces

    return text

# Apply the function to the 'Institut' column
clean_merged_df['Institut'] = clean_merged_df['Institut'].apply(process_institut_text)

In [ ]:
# Define a function to clean the 'Institut' text
def clean_institut(text):
    # Check if the input is a string
    if isinstance(text, str):
        # If the line begins with "Institut", remove it
        if text.startswith("Institut"):
            text = text.replace("Institut", "", 1).strip()  # Remove only the first occurrence of "Institut"
        
        # Replace "Institutfor" with "Institut for"
        text = re.sub(r'\bInstitutfor\b', 'Institut for', text)
    
    return text

# Apply the function to the 'Institut' column
clean_merged_df['Institut'] = clean_merged_df['Institut'].apply(clean_institut)

In [ ]:
# The dictionary for departments
departments = {
    "25 Institut for Akvatiske Ressourcer": "National Institute of Aquatic Resources",
    "27 Institut for Bioteknologi og Biomedicin": "Department of Biotechnology and Biomedicine",
    "01 Institut for Matematik og Computer Science": "Department of Applied Mathematics and Computer Science",
    "41 Institut for Byggeri og Mekanisk Teknologi": "Department of Civil and Mechanical Engineering",
    "34 Institut for Elektroteknologi og Fotonik": "Department of Electrical and Photonics Engineering",
    "47 Institut for Energikonvertering- og lagring": "Department of Energy Conversion and Storage",
    "10 Institut for Fysik": "Department of Physics",
    "23 Fødevareinstituttet": "National Food Institute",
    "26 Institut for Kemi": "Department of Chemistry",
    "28 Institut for Kemiteknik": "Department of Chemical and Biochemical Engineering",
    "42 Institut for Teknologi, Ledelse og Økonomi": "Department of Technology, Management and Economics",
    "33 Institut for Mikro- og Nanoteknologi": "National Centre for Nano Fabrication and Characterization",
    "30 Institut for Rumforskning og -teknologi": "Department of Space Research and Technology",
    "22 Institut for Sundhedsteknologi": "Department of Health Technology",
    "12 Institut for Miljø- og Ressourceteknologi": "Department of Environmental and Resource Engineering",
    "46 Institut for Vind og Energisystemer": "Department of Wind Energy",
    "24 Veterinærinstituttet": "The Veterinary Institute",
    "36 DTU Bioinformatik": "Institute for Bioinformatics",
    "88 Andre kurser": "Other courses"
}

# Extract the leading institute number
clean_merged_df['Institut_Number'] = clean_merged_df['Institut'].str.extract(r'(\d+)', expand=False)

# Map Institut "11" to "41"
clean_merged_df['Institut_Number'] = clean_merged_df['Institut_Number'].replace({'11': '41'})

# Sort the DataFrame by Year to ensure the most recent names come last
clean_merged_df = clean_merged_df.sort_values('Year', ascending=True)

# Group by 'Institut_Number' and take the last (most recent) name for each group
recent_names = clean_merged_df.groupby('Institut_Number')['Institut'].last().reset_index()

# Create a mapping of numbers to their most recent names
name_mapping = dict(zip(recent_names['Institut_Number'], recent_names['Institut']))

# Replace the Institut names in the original DataFrame with the most recent name
clean_merged_df['Institut'] = clean_merged_df['Institut_Number'].map(name_mapping)

# Now, use the departments dictionary to translate the names into English
clean_merged_df['Institut'] = clean_merged_df['Institut'].replace(departments)

# Drop the temporary 'Institut_Number' column
clean_merged_df.drop(columns=['Institut_Number'], inplace=True)

In [ ]:
# Define a function to clean the 'Hjælpemidler' text
def clean_hjælpemidler(text):
    # If the line is NaN, blank, or exactly ": i", replace it with 'Tilladt'
    if pd.isna(text) or text.strip() == "" or text.strip() == ": i":
        return "Tilladt"
    
    # Check if the input is a string
    if isinstance(text, str):
        # If the line contains "Uden hjælpemidler", replace it with "Ikke tilladt"
        if "Uden hjælpemidler" in text:
            return "Ikke tilladt"
        
        # If the line contains specific phrases, replace it with 'Tilladt'
        if ("Skriftlige hjælpemidler er tilladt" in text or 
            "Alle hjælpemidler er tilladt" in text or 
            "Karakteren for kurset vil være en helhedsvurdering" in text or
            "egne noter" in text):
            return "Tilladt"
        
        # If the line contains 'ikke tilladt', return it as it is, unless it contains the specific phrases above
        if "ikke tilladt" in text.lower() and not any(phrase in text for phrase in [
            "Skriftlige hjælpemidler er tilladt", 
            "Alle hjælpemidler er tilladt", 
            "Karakteren for kurset vil være en helhedsvurdering",
            "egne noter"]):
            return text
        
        # If the line contains 'tilladt' but not 'ikke tilladt', replace it with 'Tilladt'
        if "tilladt" in text.lower():
            return "Tilladt"
    
    # Otherwise, return the text unchanged (handles non-string types like NaN)
    return text

# Apply the function to the 'Hjælpemidler' column
clean_merged_df['Hjælpemidler'] = clean_merged_df['Hjælpemidler'].apply(clean_hjælpemidler)


In [ ]:
# Define a function to clean the 'Evalueringsform' text
def clean_evalueringsform(text):
    # Ensure the input is a string and convert it to lowercase for case-insensitive comparison
    if isinstance(text, str):
        text_lower = text.lower()
        
        # First priority: Check for "Portfolio" related phrases
        if ("afløsningsopgave" in text_lower or 
            "bedømmelse af opgave" in text_lower or 
            "bedømmelse af øvelse" in text_lower or
            "bedømmelse af rapport" in text_lower):
            return "Portfolio"
        
        # Second priority: Check for "Skriftlig eksamen" only
        if "skriftlig eksamen" in text_lower and "mundtlig eksamen" not in text_lower:
            return "Skriftlig eksamen"
        
        # Third priority: Check for "Mundtlig eksamen" only
        if "mundtlig eksamen" in text_lower and "skriftlig eksamen" not in text_lower:
            return "Mundtlig eksamen"
        
        # Fourth priority: Check if both "Skriftlig eksamen" and "Mundtlig eksamen" are present
        if "skriftlig eksamen" in text_lower and "mundtlig eksamen" in text_lower:
            return "Skriftlig og mundtlig eksamen"
        
        # Final priority: If none of the above conditions are met, return "Anden"
        return "Anden"
    
    # If the text is not a string (NaN or other), return it unchanged
    return text

# Apply the function to the 'Evalueringsform' column
clean_merged_df['Evalueringsform'] = clean_merged_df['Evalueringsform'].apply(clean_evalueringsform)


In [ ]:
# Save the cleaned DataFrame to a new CSV file
cleaned_csv_path = './cleaned_merged_course_data.csv'
clean_merged_df.to_csv(cleaned_csv_path, index=False)

# Save the cleaned DataFrame to a new Excel file
cleaned_excel_path = './cleaned_merged_course_data.xlsx'
clean_merged_df.to_excel(cleaned_excel_path, index=False)